# COMPSCI 389: Introduction to Machine Learning
# Nearest Neighbor for Regression

In this notebook we will create our first ML algorithms for regression.

As an example, we will apply the Nearest Neighbor algorithm to the GPA data set.

This Jupyter notebook is a **subset** of the content covered in 

> 5 Models, Algorithm Template, Nearest Neighbor.pdf

First, here are the import statements that we use in this notebook:

In [3]:
import pandas as pd                     # For representing data sets
from sklearn.base import BaseEstimator  # For creating our nearest neighbor model
import numpy as np                      # For representing arrays
import timeit                           # For timing different function calls
from sklearn.neighbors import KDTree    # For efficient nearest-neighbor searches (more on this below!)

Next, let's load the GPA data set and display it as a reminder of what it contains.

In [4]:
df = pd.read_csv("https://people.cs.umass.edu/~pthomas/courses/COMPSCI_389_Spring2024/GPA.csv", delimiter=',') # Read GPA.csv, assuming numbers are separated by commas
# df = pd.read_csv("data/GPA.csv", delimiter=',')
display(df)

,physics,biology,history,English,geography,literature,Portuguese,math,chemistry,gpa
0,622.60,491.56,439.93,707.64,663.65,557.09,711.37,731.31,509.80,1.33333
1,538.00,490.58,406.59,529.05,532.28,447.23,527.58,379.14,488.64,2.98333
2,455.18,440.00,570.86,417.54,453.53,425.87,475.63,476.11,407.15,1.97333
3,756.91,679.62,531.28,583.63,534.42,521.40,592.41,783.76,588.26,2.53333
4,584.54,649.84,637.43,609.06,670.46,515.38,572.52,581.25,529.04,1.58667
...,...,...,...,...,...,...,...,...,...,...
43298,519.55,622.20,660.90,543.48,643.05,579.90,584.80,581.25,573.92,2.76333
43299,816.39,851.95,732.39,621.63,810.68,666.79,705.22,781.01,831.76,3.81667
43300,798.75,817.58,731.98,648.42,751.30,648.67,662.05,773.15,835.25,3.75000
43301,527.66,443.82,545.88,624.18,420.25,676.80,583.41,395.46,509.80,2.50000


Before continuing, let's split the data set into `X` (all but the last column = 9 exam scores) and `y` (the last column = GPA) components:

In [3]:
# Split the inputs from the outputs
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
display(X)
display(y)

,physics,biology,history,English,geography,literature,Portuguese,math,chemistry
0,622.60,491.56,439.93,707.64,663.65,557.09,711.37,731.31,509.80
1,538.00,490.58,406.59,529.05,532.28,447.23,527.58,379.14,488.64
2,455.18,440.00,570.86,417.54,453.53,425.87,475.63,476.11,407.15
3,756.91,679.62,531.28,583.63,534.42,521.40,592.41,783.76,588.26
4,584.54,649.84,637.43,609.06,670.46,515.38,572.52,581.25,529.04
...,...,...,...,...,...,...,...,...,...
43298,519.55,622.20,660.90,543.48,643.05,579.90,584.80,581.25,573.92
43299,816.39,851.95,732.39,621.63,810.68,666.79,705.22,781.01,831.76
43300,798.75,817.58,731.98,648.42,751.30,648.67,662.05,773.15,835.25
43301,527.66,443.82,545.88,624.18,420.25,676.80,583.41,395.46,509.80


0        1.33333
1        2.98333
2        1.97333
3        2.53333
4        1.58667
          ...   
43298    2.76333
43299    3.81667
43300    3.75000
43301    2.50000
43302    3.16667
Name: gpa, Length: 43303, dtype: float64

Recall that our goal is to use the available data to make predictions for new data points called *queries*. These queries come with input features (e.g., exam scores), but not a label (e.g., GPA). Our goal is to create an algorithm that can predict the corresponding label given the query features.

In this notebook we will use the GPA data set to solve the regression problem of predicting student GPAs from exam scores.

## Scikit-Learn Models

Before creating and coding up our first ML algorithms, let's review a general framework for implementing ML algorithms. Specifically, we will consider a form for defining ML algorithms that is compatible with scikit-learn.

This framework relies on the term **model**. In ML, a model is a mechanism with the following abilities.
1. It can be initialized using data. This step is sometimes called "**training** the model using the data" or "**fitting** the model to the data". During this training phase the algorithm processes the provided data to pre-compute values that will be useful for making future predictions.
2. It can be given a query (one or more sets of input features), and it will produce predictions of the labels for the provided inputs. When the model makes a prediction, we say that the model is **run** or **executed**.

We will follow the scikit-learn template for representing models and ML algorithms. Creating a new ML algorithm means implementing the following functions:
1. `__init__`: A constructor that can be used to set hyperparameters that change the behavior of the algorithm.
2. `fit(self, X, y)`: The function for fitting the model to the data (training the model given the data). This function is called to allow the ML algorithm to pre-process the data so that it can more quickly respond to future queries.
    - `X`: A 2D array-like structure (e.g., DataFrame) representing the features. Each row is a point and each column is a feature.
    - `y`: A 1D array-like structure (e.g., Series) representing the target values.
    - **Returns**: This function returns `self` to simplify chaining together operations.
3. `predict(self, X)`: The function for producing predictions given queries.
    - `X`: A 2D array-like structure representing the data for which predictions are to be made. Each row in `X` is a sample, and each column is a feature.
    - **Returns**: A numpy array of predicted labels/values.

For example, here is a template of the code to create a new ML algorithm that is compatible with scikit-learn:

In [4]:
class CustomMLAlgorithm(BaseEstimator):
    def __init__(self, param1=1, param2=2):
        # Initialization code
        self.param1 = param1                        # If you have hyperparameters of your algorithm, they are set here like param1 and param2
        self.param2 = param2

    def fit(self, X, y):
        # Training code
        # Implement your training algorithm here
        return self

    def predict(self, X):
        # Prediction code
        # Implement your prediction algorithm here
        return np.zeros(len(X))                     # For now we just return all zeroes

We can then create a model with:

`model = CustomMLAlgorithm()`

If `X` is a DataFrame containing the input features and `y` is a Series containing the resulting labels, we can train the model with:

`model.fit(X,y)`

If `query` is a DataFrame containing the inputs for which we would like to predict the labels, we can get the predictions with:

`predictions = model.predict(query)`

## Nearest Neighbor 

*Nearest Neighbor* (NN) is a particularly simple yet effective ML algorithm based on the core idea:

> When presented with a query, find the data point (row) that is most similar to the query and give the label associated with this most-similar point as the prediction.

We map this to the scikit-learn functions by having `fit` store the data and `predict` handle all of the processing. Predict does the following:

1. Loop over each row in the training data, computing the Euclidean distance between the query and the row.
2. Find the rows with the smallest distance to the query feature vector (if there are ties, there can be more than one).
2. Create an array holding the labels from these rows.
3. Return an arbitrary element of the array.

Here is a naive implementation of NN:

In [5]:
class NaiveNearestNeighbor(BaseEstimator):
    def fit(self, X, y):
        # Convert X and y to NumPy arrays if they are DataFrames. 
        # This makes fit compatible with numpy arrays or DataFrames
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        # Store the training data and labels.
        self.X_data = X
        self.y_data = y
        return self

    def predict(self, X):
        # Convert X to a NumPy array if it's a DataFrame
        if isinstance(X, pd.DataFrame):
            X = X.values

        # We will iteratively load predictions, so it starts empty
        predictions = []
        
        # Loop over rows in the query
        for x in X:
            # Compute distances from x to all points in X_data. We combine the following steps into one line:
            # differences = self.X_data - x                                   # Subtract x from each row of X
            # squared_differences = differences ** 2                          # Square the differences
            # sum_squared_differences = np.sum(squared_differences, axis=1)   # Sum each row, giving one number per column
            # distances = np.sqrt(sum_squared_differences)                    # Take the square root of the sum_squared_differences, to get the Euclidean distance.
            distances = np.sqrt(np.sum((self.X_data - x) ** 2, axis=1))
            
            # Find the nearest neighbors (handling ties)
            min_distance = np.min(distances)
            nearest_neighbors = np.where(distances == min_distance)[0]      # np.where returns a tuple of arrays, one for each dimension. E.g., if we gave a 2d array, [0] would be the row index and [1] would be the col index. Distances is 1-D, so there is only one element in the tuple here, hence we pass [0]
            nearest_label = self.y_data[nearest_neighbors[0]]               # You could return a random element of this array. For now we return the first element.

            # Append this label to predictions
            predictions.append(nearest_label)

        # Return the array of predictions we have created
        return np.array(predictions)

Let's apply this to the GPA data set, which we already loaded into a DataFrame `df`:

In [6]:
# Create the Nearest Neighbor Model
model = NaiveNearestNeighbor()

# Call fit to train the model (in this case, just store the data set)
model.fit(X,y)

# Create two query points (in reality these would be new applicants)
query = df.head(2).iloc[:,:-1]

# Get predictions for the query points
predictions = model.predict(query)
print(predictions)

[1.33333 2.98333]


It works! Well, it produces predictions. Soon we will investigate how *good* these predictions are. First, let's make our algorithm more efficient.

### Optimizing Nearest Neighbor Search

The current predict method in our Nearest Neighbor algorithm loops over the entire dataset for each query point, which is inefficient for large datasets.

Instead, we will use data structures designed to make finding nearest neighbors efficient: K-D Trees (or KD Trees) and Ball Trees

- **Purpose**: To store points and enable efficient search for the closest points to a query.
- **K-D Trees**: Effective for low-dimensional data, but performance decreases with higher dimensions.
- **Ball Trees**: Better suited for higher-dimensional spaces.

### Implementation with Scikit-Learn

We will update our algorithm to use a K-D Tree, leveraging sklearn.neighbors from scikit-learn. This module provides optimized implementations of these data structures, balancing preprocessing time, search speed, and accuracy (exact vs. approximate nearest neighbors).

In [ ]:
class NearestNeighbor(BaseEstimator):
    def fit(self, X, y):
        # Convert X and y to NumPy arrays if they are DataFrames. 
        # This makes fit compatible with numpy arrays or DataFrames
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        # Store the training data and labels.
        self.X_data = X
        self.y_data = y
        
        # Create a KDTree for efficient nearest neighbor search
        self.tree = KDTree(X)

        return self

    def predict(self, X):
        # Convert X to a NumPy array if it's a DataFrame
        if isinstance(X, pd.DataFrame):
            X = X.values

        # Query the tree for the nearest neighbors of all points in X.
        # ind will be a 2D array where ind[i,j] is the index of the
        # j'th nearest point to the i'th row in X.
        dist, ind = self.tree.query(X, k=1) 

        # Extract the nearest labels.
        # ind[:,0] are the indices of the nearest neighbors to each
        # query (each row in x))
        return self.y_data[ind[:,0]]

Let's see if this gives the same answer as our naive implementation:

In [9]:
model = NearestNeighbor()
model.fit(X,y)
predictions = model.predict(query)
print(predictions)

[1.33333 2.98333]


Great, they're the same! Note that the two implementations won't necessarily produce the same output when there is more than one nearest point. Next, let's compare their runtimes. Note that this comparison may vary with the size, dimension, and sparsity of the data set.

Note that `timeit` times how long it takes to run a block of code many times.

In the code below, `lambda:` is a way of defining a function without naming it using `def`. The format is:

> lambda arguments: expression

e.g.:
```
sum_lambda = lambda x, y: x + y
print(sum_lambda(5, 3))  # This will output 8
```
We use `lambda` here to wrap the argument (the model used) into the function timed using `timeit`.

In [10]:
def run_model(model_class):
    model = model_class()
    model.fit(X, y)
    predictions = model.predict(query)
    return predictions

# Number of trials
numTrials = 100

# Time the NaiveNearestNeighbor
time_naive = timeit.timeit(lambda: run_model(NaiveNearestNeighbor), number=numTrials)
print(f"Average runtime for NaiveNearestNeighbor: {time_naive / numTrials} seconds")

# Time the NearestNeighbor
time_efficient = timeit.timeit(lambda: run_model(NearestNeighbor), number=numTrials)
print(f"Average runtime for NearestNeighbor: {time_efficient / numTrials} seconds")


Average runtime for NaiveNearestNeighbor: 0.0033271799999056383 seconds
Average runtime for NearestNeighbor: 0.08059094200027175 seconds


**Question**: Why do you think our naive algorithm was faster?

It could be that the overhead cost of building the KD-tree is not worth it when only running two queries. Let's run more queries:

In [11]:
# Let's run 5,000 of the points through as queries
query = df.iloc[1:5000,:-1]

# 100 trials is now quite slow. Let's run 1. You could increase this to get a more accurate estimate of runtime.
numTrials = 1

# Time the NaiveNearestNeighbor
time_naive = timeit.timeit(lambda: run_model(NaiveNearestNeighbor), number=numTrials)
print(f"Average runtime for NaiveNearestNeighbor: {time_naive / numTrials} seconds")

# Time the NearestNeighbor
time_efficient = timeit.timeit(lambda: run_model(NearestNeighbor), number=numTrials)
print(f"Average runtime for NearestNeighbor: {time_efficient / numTrials} seconds")

Average runtime for NaiveNearestNeighbor: 8.219239899946842 seconds
Average runtime for NearestNeighbor: 0.09280269994633272 seconds


It should now be clear that data structures like K-D Trees can speed up nearest neighbor searches when we run many queries, but there is an overhead cost associated with constructing the K-D Tree that makes it less efficient when running a single query (or small number of queries).